In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from keras.constraints import maxnorm
from tensorflow.keras import backend
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
import numpy as np

Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
image_height = X_train.shape[1]
image_width = X_train.shape[2]
num_pix = image_height * image_width 

In [3]:
X_train = backend.cast_to_floatx(X_train)
X_test = backend.cast_to_floatx(X_test)

In [4]:
X_train /= 255.
X_test /= 255.

In [5]:
original_y_train = y_train
original_y_test = y_test

In [6]:
number_of_classes = 1 + max(np.append(y_train, y_test))
y_train = to_categorical(y_train, number_of_classes)
y_test = to_categorical(y_test, number_of_classes)

In [7]:
X_train = X_train.reshape([X_train.shape[0], num_pix])
X_test = X_test.reshape([X_test.shape[0], num_pix])

In [8]:
def my_model(num_layers=2, neurons_per_layer=32,
            dropout_ratio = 0.2, optimizer='adam'):
    
    model = Sequential()
    model.add(Dense(neurons_per_layer,
                   input_shape=[num_pix],
                   activation = 'relu',
                   kernel_constraint=maxnorm(3)))
    model.add(Dropout(dropout_ratio))
    
    for i in range(num_layers-1):
        model.add(Dense(neurons_per_layer, activation='relu',
                       kernel_constraint=maxnorm(3)))
        model.add(Dropout(dropout_ratio))
        
        model.add(Dense(number_of_classes,
                       kernel_initializer='normal',
                       activation='softmax'))
        model.compile(loss='categorical_crossentropy',
                     optimizer=optimizer, metrics=['accuracy'])
        
        return model
    

In [9]:
kc_model = KerasClassifier(build_fn=my_model,
                          num_layers=2, neurons_per_layer=32,
                          optimizer='adam',
                          epochs=100, batch_size=256, verbose=0)

In [10]:
param_grid = dict(num_layers=[2, 3, 4],
                 neurons_per_layer=[20, 30, 40],
                 optimizer=['adam', 'adadelta'])
grid_search = GridSearchCV(estimator=kc_model,
                          param_grid=param_grid, verbose=2)
search_result = grid_search.fit(X_train, original_y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] neurons_per_layer=20, num_layers=2, optimizer=adam ..............
[CV]  neurons_per_layer=20, num_layers=2, optimizer=adam, total=  49.9s
[CV] neurons_per_layer=20, num_layers=2, optimizer=adam ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   49.9s remaining:    0.0s


[CV]  neurons_per_layer=20, num_layers=2, optimizer=adam, total=  50.4s
[CV] neurons_per_layer=20, num_layers=2, optimizer=adam ..............
[CV]  neurons_per_layer=20, num_layers=2, optimizer=adam, total=  50.5s
[CV] neurons_per_layer=20, num_layers=2, optimizer=adadelta ..........
[CV]  neurons_per_layer=20, num_layers=2, optimizer=adadelta, total=  48.5s
[CV] neurons_per_layer=20, num_layers=2, optimizer=adadelta ..........
[CV]  neurons_per_layer=20, num_layers=2, optimizer=adadelta, total=  49.5s
[CV] neurons_per_layer=20, num_layers=2, optimizer=adadelta ..........
[CV]  neurons_per_layer=20, num_layers=2, optimizer=adadelta, total=  51.8s
[CV] neurons_per_layer=20, num_layers=3, optimizer=adam ..............
[CV]  neurons_per_layer=20, num_layers=3, optimizer=adam, total=  48.5s
[CV] neurons_per_layer=20, num_layers=3, optimizer=adam ..............
[CV]  neurons_per_layer=20, num_layers=3, optimizer=adam, total=  48.9s
[CV] neurons_per_layer=20, num_layers=3, optimizer=adam ..

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 51.0min finished


In [25]:
search_result.cv_results_['params']

[{'neurons_per_layer': 20, 'num_layers': 2, 'optimizer': 'adam'},
 {'neurons_per_layer': 20, 'num_layers': 2, 'optimizer': 'adadelta'},
 {'neurons_per_layer': 20, 'num_layers': 3, 'optimizer': 'adam'},
 {'neurons_per_layer': 20, 'num_layers': 3, 'optimizer': 'adadelta'},
 {'neurons_per_layer': 20, 'num_layers': 4, 'optimizer': 'adam'},
 {'neurons_per_layer': 20, 'num_layers': 4, 'optimizer': 'adadelta'},
 {'neurons_per_layer': 30, 'num_layers': 2, 'optimizer': 'adam'},
 {'neurons_per_layer': 30, 'num_layers': 2, 'optimizer': 'adadelta'},
 {'neurons_per_layer': 30, 'num_layers': 3, 'optimizer': 'adam'},
 {'neurons_per_layer': 30, 'num_layers': 3, 'optimizer': 'adadelta'},
 {'neurons_per_layer': 30, 'num_layers': 4, 'optimizer': 'adam'},
 {'neurons_per_layer': 30, 'num_layers': 4, 'optimizer': 'adadelta'},
 {'neurons_per_layer': 40, 'num_layers': 2, 'optimizer': 'adam'},
 {'neurons_per_layer': 40, 'num_layers': 2, 'optimizer': 'adadelta'},
 {'neurons_per_layer': 40, 'num_layers': 3, 'opt

In [26]:
search_result.cv_results_['mean_test_score']

array([0.94476666, 0.37196667, 0.94546666, 0.34173334, 0.94731669,
       0.39698334, 0.95768334, 0.47233334, 0.95694999, 0.45550001,
       0.95861667, 0.44076667, 0.96403333, 0.55585001, 0.96498332,
       0.58835   , 0.96436665, 0.56681667])

In [30]:
best_index = np.argmax(search_result.cv_results_['mean_test_score'])
print('Best set of parameters:\n index {}\n{}\n'.format(best_index,
                                                   search_result.cv_results_['params'][best_index]))

Best set of parameters:
 index 14
{'neurons_per_layer': 40, 'num_layers': 3, 'optimizer': 'adam'}

